<a href="https://colab.research.google.com/github/rgukhui/cm4709/blob/main/Lab07_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CM4709 Computer Vision
#Lab 07 Image Segmentation

##Aims
* Use PointRend to perform image segmentation.

Note: The original lab used [Mask RCNN](https://github.com/matterport/Mask_RCNN).
However, as software have evolved, the current library versions do not seem to work with Mask RCNN and the older version of Tensorflow is not available in Colab anymore.
As a result, we changed to use [PointRend](https://arxiv.org/abs/1912.08193). The whole idea is to illustrate how to do image segmentation using in an
existing library. This does not change in using a different network architecture.

This lab is based on the code given by [PixelLib](https://github.com/ayoolaolafenwa/PixelLib).

##Mounting GoogleDrive to Colab Runtime

As usual, you will need some testing images. It is faster to upload them to GoogleDrive and then mount it to your Colab runtime.

In the following code, I use the path `/content/drive` as the mounting point as Colab also has a convenient button mounting to this path.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Testing Images

You are advised to organise your files in GoogleDrive. e.g. Create a `Lab07` folder in GoogleDrive to store the notebook file of Lab 07, and a `Lab07/data` folder for the images.

1. Download the testing images in Moodle. Optionally, use your own testing images.
1. Upload testing images to your GoogleDrive space.

The following code shows a testing image using OpenCV.
* Note: Modify the filename and image path accordingly.


In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

#image filename and path
#
filename='test.jpg'
imagePath='/content/drive/MyDrive/cm4709/Lab07/data'
imageFile=imagePath+'/'+filename

#show image
#
img=cv.imread(imageFile)
plt.figure(figsize=(10,10))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

##Install PixelLib

[PixelLib](https://github.com/ayoolaolafenwa/PixelLib) provides some handy libraries to do image segmentation. The current version has updated to used Pytorch instead of Tensorflow.

The following shell commands install Pixellib:


In [ ]:
!pip3 install pixellib

##The Pre-trained Model/Weights File

Next, we need the pre-trained model's weights.


The following shell command downloads the weights file into the default folder of your Colab runtime:

In [ ]:
!wget https://github.com/ayoolaolafenwa/PixelLib/releases/download/0.2.0/pointrend_resnet50.pkl

##Creating the Model

You are now ready to create a PointRend model to do image segmentation.
In the following code:
* Modify the path the image if needed.
* Modify the path to the weights file if needed.
* The function `segmentImage(...)` takes a parameter `output_image_name`. The image segmentation output will be written to this file. Modify this if you need to.

In [ ]:
import pixellib
from pixellib.torchbackend.instance import instanceSegmentation

#
#path to input image
#
filename='test.jpg'
imagePath='/content/drive/MyDrive/cm4709/Lab07/data'
imageFile=imagePath+'/'+filename

#
#path to output result
#
output='output.jpg'
outputPath='/content'
outputFile=outputPath+'/'+output

segment_image = instanceSegmentation()
segment_image.load_model("pointrend_resnet50.pkl")
segment_image.segmentImage(imageFile, output_image_name=outputFile)

##Seeing the Segmentation Result

PointRend saves the result to the image file you specified.
Alternatively, the following code displays the output image:

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

#
#path to output result
#
output='output.jpg'
outputPath='/content'
outputFile=outputPath+'/'+output

#
#show segementation result
#
img=cv.imread(outputFile)
plt.figure(figsize=(10,10))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))


##Other Functions

[PixelLib](https://pixellib.readthedocs.io/en/latest/) has many handy functions built-in.
You can see the documentation [here](https://pixellib.readthedocs.io/en/latest/Image_instance.html).
For example, you can show the bounding boxes of the detected segments:


In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import pixellib
from pixellib.instance import instance_segmentation

#
#path to input image
#
filename='test2.jpg'
imagePath='/content/drive/MyDrive/cm4709/Lab07/data'
imageFile=imagePath+'/'+filename

#
#path to output result
#
output='output.jpg'
outputPath='/content'
outputFile=outputPath+'/'+output

#
#path to weights file
#
weightFile='/content/pointrend_resnet50.pkl'

#
#perform image segmentation
#
segment_image = instanceSegmentation()
segment_image.load_model(weightFile)
segment_image.segmentImage(imageFile,show_bboxes=True, output_image_name=outputFile)

#
#show segementation result
#
img=cv.imread(outputFile)
plt.figure(figsize=(10,10))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

You can also extract and save the detected segments:

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import pixellib
from pixellib.torchbackend.instance import instanceSegmentation

#
#path to output result
#
output='output.jpg'
outputPath='/content'
outputFile=outputPath+'/'+output

#
#path to weights file
#
weightFile='/content/pointrend_resnet50.pkl'

#
#perform image segmentation
#extract and save segments
#
segment_image = instanceSegmentation()
#segment_image.load_model(weightFile)
segment_image.load_model("pointrend_resnet50.pkl")
segment_image.segmentImage(imageFile, output_image_name=outputFile,extract_segmented_objects=True,save_extracted_objects=True)

Extracted segments are saved to the home of your Colab runtime with filenames `segmented_object_...`.
The following code shows all of them:

In [ ]:
import os
import fnmatch

segmentPath='.'
pattern='segmented_object_*.jpg'
segmentFiles=[]

#collect segment files full paths
#
allFiles=os.listdir(segmentPath)
for file in allFiles:
  if fnmatch.fnmatch(file,pattern):
    segmentFiles.append(segmentPath+'/'+file)

total=len(segmentFiles) #no. of files
print('No. of files: ',total)

colCount=10
rowCount=int(total/colCount)+1
plt.figure(figsize=(colCount*1.5,rowCount*1.5))
i=1

for file in segmentFiles:
  img=cv.imread(file)
  cvtImg=cv.cvtColor(img, cv.COLOR_BGR2RGB)
  plt.subplot(rowCount,colCount,i)
  plt.imshow(cvtImg)
  i=i+1

If you need to remove all `segemented_object_*.jpg` files between different runs, the following code will be handy:

In [6]:
#remove all extracted segment files
#
import os
import fnmatch

segmentPath='.'
pattern='segmented_object_*.jpg'

#collect segment files full paths
#
allFiles=os.listdir(segmentPath)
for file in allFiles:
  if fnmatch.fnmatch(file,pattern):
    os.remove(segmentPath+'/'+file)